# Connect4 DQN model
By LaughingSkull 
as new RL agent for my game COnnect-4: https://www.laughingskull.org/Games/Connect4/Connect4.php


## DQN (Deep Q-Network)

* A value-based reinforcement learning method.
* Uses a neural network to approximate the Q-function: estimates the expected future rewards for taking actions in given states.
* Learns via Q-learning: updates the Q-values using the Bellman equation.
* Typically uses techniques like experience replay and target networks to stabilize training.

### version log    

* 0.8.0 - start with new convolutional part<br>
    * keeping L2 regularization
* 0.9.0 - extension to shallow training
* 0.10.0 - extension to L3 training
    * NO PRUNE:
        * worse 
    * SHARPer PRUNE
    * extend phases
        * MixedR12: no improvement; &cross;
        * Shallow
        * Fixed2
        * Variable23
        * Variable3
* 0.11.0 - recover fixed 2
    * corrected reward leaking bug
    * corrected draw bug
* 0.12.0 refactoring and restarting
* 0.13 restarting again
    * strategy weights djustment
* 0.14 new restart from scratch
    * intermediate phases 

## Links, learning from
[https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html](https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)
<br>
[https://pettingzoo.farama.org/tutorials/agilerl/DQN/](https://pettingzoo.farama.org/tutorials/agilerl/DQN/)
<br>
### Other helpful links

<br>[https://medium.com/@vishwapatel214/building-a-connect-4-game-bot-with-deep-learning-models-dbcd019d8967](https://medium.com/@vishwapatel214/building-a-connect-4-game-bot-with-deep-learning-models-dbcd019d8967)
<br>[https://codebox.net/pages/connect4#:~:text=This%20requires%20a%20lot%20of%20work%20up%2Dfront,possible%20action%20at%20each%20step%20is%20impractical.](https://codebox.net/pages/connect4#:~:text=This%20requires%20a%20lot%20of%20work%20up%2Dfront,possible%20action%20at%20each%20step%20is%20impractical.)
<br>[https://medium.com/advanced-machine-learning/deep-learning-meets-board-games-creating-a-connect-4-ai-using-cnns-and-vits-89c8cdab0041](https://medium.com/advanced-machine-learning/deep-learning-meets-board-games-creating-a-connect-4-ai-using-cnns-and-vits-89c8cdab0041)
<br>[https://medium.com/@piyushkashyap045/understanding-dropout-in-deep-learning-a-guide-to-reducing-overfitting-26cbb68d5575#:~:text=Choosing%20Dropout%20Rate:%20Common%20dropout,is%20better%20for%20simpler%20models.](https://medium.com/@piyushkashyap045/understanding-dropout-in-deep-learning-a-guide-to-reducing-overfitting-26cbb68d5575#:~:text=Choosing%20Dropout%20Rate:%20Common%20dropout,is%20better%20for%20simpler%20models.)
<br>
[https://medium.com/oracledevs/lessons-from-alphazero-connect-four-e4a0ae82af68](https://medium.com/oracledevs/lessons-from-alphazero-connect-four-e4a0ae82af68)
<br>
[https://docs.agilerl.com/en/latest/tutorials/pettingzoo/dqn.html](https://docs.agilerl.com/en/latest/tutorials/pettingzoo/dqn.html)
<br>



## Import dependecies and recheck installation

In [ ]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import pprint;
import pandas as pd
import os
import json
from IPython.display import display, clear_output

print("All dependencies imported successfully.")
print("torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available. Using CPU.")


### Fixed Random seeds

In [ ]:
SEED = 666
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.environ["PYTHONHASHSEED"] = str(SEED)

## Custom imports

In [ ]:
from C4.connect4_env import Connect4Env
from C4.connect4_lookahead import Connect4Lookahead
from DQN.training_phases_config import TRAINING_PHASES
from DQN.opponent_action import get_opponent_action
from DQN.DQN_replay_memory import ReplayMemory
from DQN.dqn_model import DQN
from DQN.dqn_agent import DQNAgent
from DQN.get_phase import get_phase
from C4.connect4_board_display import Connect4_BoardDisplayer

## Training session name and settings

In [ ]:
lookahead_depth = 7  # prophet = 7

num_episodes = 5000

batch_size = 64
target_update_interval = 10
log_every_x_episode = 100
tag = "Mixed_R1122 0.1, 0.4, 0.5, 0, 0, 0 E04b MP04b"

TRAINING_SESSION = f"{tag}-{num_episodes}"
begin_start_time = time.time()
LOG_DIR ="Logs/DQN/"
MODEL_DIR ="Models/DQN/"
PLOTS = "Plots/DQN/"
PRUNE = True
print("Started training session", TRAINING_SESSION)

### Model overview

In [ ]:
_model = DQN()
summary(_model, input_size=(1, 6, 7))  # batch height, width

## Training loop - DQN against lookahead opponent (Prophet-style)

### Training config

In [ ]:
# --- Save training configuration to Excel ---

config_log = {
    "TRAINING_PHASES": TRAINING_PHASES,  # Updated structure
    "lookahead_depth": lookahead_depth,
    "num_episodes": num_episodes,
    "batch_size": batch_size,
    "target_update_interval": target_update_interval
}

# Flatten nested structure for Excel output
flat_config = []
for section, values in config_log.items():
    if isinstance(values, dict):
        for phase_name, params in values.items():
            if isinstance(params, dict):
                for param_key, param_value in params.items():
                    flat_config.append({
                        "Section": section,
                        "Phase": phase_name,
                        "Key": param_key,
                        "Value": str(param_value)
                    })
            else:
                flat_config.append({
                    "Section": section,
                    "Phase": phase_name,
                    "Key": "",
                    "Value": str(params)
                })
    else:
        flat_config.append({
            "Section": section,
            "Phase": "",
            "Key": "",
            "Value": str(values)
        })

# Create DataFrame
config_df = pd.DataFrame(flat_config)

# Save to Excel
excel_path = f"{LOG_DIR}DQN-{TRAINING_SESSION}_training_config.xlsx"
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    config_df.to_excel(writer, sheet_name='config', index=False)


### Training loop

In [ ]:
summary_stats = {}  # store stats
env = Connect4Env()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)
agent = DQNAgent(device=device)

reward_history = []
win_history = []
epsilon_history = []
win_count = loss_count = draw_count = 0
phase = "Random"
strategy_weights = []
start_time = time.time()

with tqdm(total=num_episodes, desc="Training Episodes") as pbar:
    for episode in range(1, num_episodes + 1):
        state = env.reset()
        total_reward = 0
        done = False
        final_result = None  # 1 = win, -1 = loss, 0.5 = draw

        while not done:
            valid_actions = env.available_actions()
            new_phase, strategy_weights, epsilon, memory_prune = get_phase(episode)

            # react to phase change
            if new_phase != phase:
                agent.epsilon = epsilon
                if PRUNE and memory_prune:
                    agent.memory.prune(memory_prune)
                phase = new_phase

            player_before_move = env.current_player  # <- track agent identity
            action = agent.act(state, valid_actions, player=env.current_player, depth=lookahead_depth, strategy_weights=strategy_weights)
            next_state, reward, done = env.step(action)

            if done:
                if env.winner == player_before_move:
                    final_result = 1  # agent won
                elif env.winner == -player_before_move:
                    final_result = -1  # opponent won
                elif env.winner == 0:
                    final_result = 0.5  # draw
                else:
                    final_result = None

                if final_result == 1:
                    agent.remember(state, action, reward, next_state, done)
            else:
                opp_action = get_opponent_action(env, agent, episode, next_state, player=-1, depth=lookahead_depth)
                next_state, opp_reward, done = env.step(opp_action)
                if done:
                    if env.winner == player_before_move:
                        final_result = 1
                    elif env.winner == -player_before_move:
                        final_result = -1
                    elif env.winner == 0:
                        final_result = 0.5
                    else:
                        final_result = None
                else:
                    agent.remember(state, action, reward, next_state, done)

            agent.replay(batch_size)
            state = next_state
            total_reward += reward

        epsilon_history.append(agent.epsilon)
        reward_history.append(total_reward)

        if final_result == 1:
            win_count += 1
            win_history.append(1)
        elif final_result == -1:
            loss_count += 1
            win_history.append(0)
        elif final_result == 0.5:
            draw_count += 1
            win_history.append(0)
        else:
            raise ValueError("Invalid final_result — env.winner was not set correctly.") # should not happen anymore

        if episode % target_update_interval == 0:
            agent.update_target_model()

        if episode % 10 == 0:
            avg_reward = np.mean(reward_history[-25:])
            pbar.set_postfix(avg_reward=f"{avg_reward:.2f}",
                             epsilon=f"{agent.epsilon:.3f}",
                             wins=win_count,
                             losses=loss_count,
                             draws=draw_count,
                             phase=phase)

            clear_output(wait=True)
            
            if done: Connect4_BoardDisplayer.display_board(next_state)

            # live plots   
            fig, ax = plt.subplots(3, 1, figsize=(10, 9), sharex=True)

            ax[0].plot(reward_history, label='Reward')
            if len(reward_history) >= 25:
                avg = np.convolve(reward_history, np.ones(25) / 25, mode='valid')
                ax[0].plot(range(24, len(reward_history)), avg, label='25-ep Moving Avg', linewidth=2)
            ax[0].set_ylabel('Total Reward')
            ax[0].legend()
            ax[0].grid(True)

            if len(win_history) >= 25:
                win_avg = np.convolve(win_history, np.ones(25) / 25, mode='valid')
                ax[1].plot(range(24, len(win_history)), win_avg, label='Win Rate (25 ep)', color='green')
            ax[1].set_ylabel('Win Rate')
            if len(ax[1].lines) > 0:
                ax[1].legend()
            ax[1].grid(True)

            ax[2].plot(epsilon_history, label='Epsilon', color='orange')
            ax[2].set_xlabel('Episode')
            ax[2].set_ylabel('Epsilon')
            ax[2].legend()
            ax[2].grid(True)

            fig.suptitle(f"Episode {episode} — Phase: {phase} | Wins: {win_count}, Losses: {loss_count}, Draws: {draw_count} | ε={agent.epsilon:.3f}")
            plt.tight_layout(rect=[0, 0, 1, 0.95])

            for name, meta in TRAINING_PHASES.items():
                transition_ep = meta["length"]
                if transition_ep is not None and transition_ep <= episode:
                    for axis in ax:
                        axis.axvline(transition_ep, color='black', linestyle='dotted', linewidth=1)
                        axis.text(transition_ep + 2, axis.get_ylim()[1] * 0.95, name,
                                  rotation=90, va='top', ha='left', fontsize=8)

            display(fig)
            plt.close()

        if episode % log_every_x_episode == 0:
            recent_rewards = reward_history[-25:] if len(reward_history) >= 25 else reward_history
            recent_win_rate = np.mean(win_history[-25:]) if len(win_history) >= 25 else np.mean(win_history)
            summary_stats[episode] = {
                "phase": phase,
                "strategy_weights": strategy_weights.copy(),
                "epsilon": agent.epsilon,
                "wins": win_count,
                "losses": loss_count,
                "draws": draw_count,
                "avg_reward_25": round(np.mean(recent_rewards), 2),
                "win_rate_25": round(recent_win_rate, 2)
            }
            
        pbar.update(1)

end_time = time.time()
elapsed = end_time - start_time
print(f"\nTraining completed in {elapsed/60:.1f} minutes ({elapsed / num_episodes:.2f} s/episode)")

# --- Save final Win Rate plot ---
final_win_fig, final_win_ax = plt.subplots(figsize=(10, 4))

if len(win_history) >= 25:
    win_avg = np.convolve(win_history, np.ones(25)/25, mode='valid')
    final_win_ax.plot(range(24, len(win_history)), win_avg, label='Win Rate (25 ep)', color='green')

final_win_ax.set_title("Final Win Rate Over Training")
final_win_ax.set_xlabel("Episode")
final_win_ax.set_ylabel("Win Rate")
final_win_ax.grid(True)
final_win_ax.legend()

for name, meta in TRAINING_PHASES.items():
    transition_ep = meta["length"]
    if transition_ep is not None and transition_ep <= len(win_history):
        final_win_ax.axvline(transition_ep, color='black', linestyle='dotted', linewidth=1)
        final_win_ax.text(
            transition_ep + 2,
            final_win_ax.get_ylim()[1] * 0.95,
            name,
            rotation=90,
            va='top',
            ha='left',
            fontsize=8
        )


final_win_fig.savefig(f"{PLOTS}DQN-{TRAINING_SESSION}_final_winrate.png")
plt.close(final_win_fig)
print(f"Win rate plot saved to {PLOTS}DQN-{TRAINING_SESSION}_final_winrate.png")


In [ ]:
print(f"\nSummary stats (every {log_every_x_episode} episodes):")
#pprint.pprint(summary_stats)
pd.DataFrame.from_dict(summary_stats, orient='index').to_excel(f"{LOG_DIR}DQN-{TRAINING_SESSION}-training_summary.xlsx", index=True)

In [ ]:
plt.plot(reward_history)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("DQN Training Progress")
plt.grid(True)
plt.show()

In [ ]:
window = 50
smoothed = [np.mean(reward_history[max(0, i-window):i+1]) for i in range(len(reward_history))]
plt.plot(smoothed)
plt.show()

In [ ]:
window = 250
smoothed = [np.mean(reward_history[max(0, i - window):i + 1]) for i in range(len(reward_history))]

final_reward_fig, final_reward_ax = plt.subplots(figsize=(10, 5))
final_reward_ax.plot(smoothed, label=f"Smoothed Reward (window={window})", color='blue')

# --- Add phase transitions ---
for name, meta in TRAINING_PHASES.items():
    ep = meta["length"]
    if ep is not None and ep <= len(reward_history):
        final_reward_ax.axvline(x=ep, color='black', linestyle='dotted', linewidth=1)
        final_reward_ax.text(ep + 5, max(smoothed) * 0.95, name,
                             rotation=90, va='top', ha='left', fontsize=8)

final_reward_ax.set_title("Smoothed Reward Over Episodes")
final_reward_ax.set_xlabel("Episode")
final_reward_ax.set_ylabel("Smoothed Reward")
final_reward_ax.legend()
final_reward_ax.grid(True)
final_reward_fig.tight_layout()

# --- Show plot ---
plt.show()

# --- Save to file ---
final_reward_fig.savefig(f"{PLOTS}DQN-{TRAINING_SESSION}_final_reward_smoothed.png")
plt.close(final_reward_fig)
print(f"Smoothed reward plot saved to {PLOTS}DQN-{TRAINING_SESSION}_final_reward_smoothed.png")


## Save model

In [ ]:
timestamp = time.strftime("%Y%m%d-%H%M%S")
model_path = f"{MODEL_DIR}{TRAINING_SESSION}_Connect4 dqn_model_{timestamp} episodes-{num_episodes} lookahead-{lookahead_depth}.pt"
default_model_path = "Connect4 DQN model.pt"

torch.save(agent.model.state_dict(), model_path)
torch.save(agent.model.state_dict(), default_model_path)
print(f"Model saved to {model_path}")


## Load model

In [ ]:
agent = DQNAgent(device=device)  # Fresh agent instance
state_dict = torch.load(default_model_path, map_location=device, weights_only=True)
agent.model.load_state_dict(state_dict)
agent.update_target_model()
agent.epsilon = 0.0  # Fully greedy — no exploration
print("✅ Trained model loaded and ready for evaluation.")


# Evaluation

In [ ]:
# === EVALUATION CONFIGURATION ===
evaluation_opponents = {
    "Random": 100,
    "Lookahead-1": 100,
    "Lookahead-2": 100,
    "Lookahead-3": 25,
    # "Lookahead-5": 10, # too soon to test
    # "Lookahead-7": 5  # too soon to test
}

# === Evaluation Loop ===
evaluation_results = {}
Lookahead = Connect4Lookahead()
start_time = time.time()

for label, num_games in evaluation_opponents.items():
    wins = losses = draws = 0
    depth = int(label.split("-")[1]) if label.startswith("Lookahead") else None

    with tqdm(total=num_games, desc=f"Opponent: {label}") as pbar:
        for _ in range(num_games):
            state = env.reset()
            done = False
            agent_first = random.choice([True, False])

            while not done:
                if (env.current_player == 1 and agent_first) or (env.current_player == -1 and not agent_first):
                    valid_actions = env.available_actions()
                    action = agent.act(state, valid_actions, player=env.current_player, depth=lookahead_depth, strategy_weights=None)
                else:
                    if label == "Random":
                        action = random.choice(env.available_actions())
                    else:
                        board = np.array(state)
                        action = Lookahead.n_step_lookahead(board, env.current_player, depth=depth)
            
                state, reward, done = env.step(action)


            # Determine winner
            winner = -env.current_player
            if winner == 1:
                if agent_first:
                    wins += 1
                else:
                    losses += 1
            elif winner == -1:
                if not agent_first:
                    wins += 1
                else:
                    losses += 1
            else:
                draws += 1

            pbar.update(1)

    evaluation_results[label] = {
        "wins": wins,
        "losses": losses,
        "draws": draws,
        "win_rate": round(wins / num_games, 2),
        "loss_rate": round(losses / num_games, 2),
        "draw_rate": round(draws / num_games, 2)
    }


end_time = time.time()
elapsed = end_time - start_time
print(f"Evaluation completed in {elapsed/60:.1f} minutes")

# === Print Summary ===
print("\n📊 Evaluation Summary:")
for label, stats in evaluation_results.items():
    print(f"{label}: {stats['wins']}W / {stats['losses']}L / {stats['draws']}D → "
          f"Win Rate: {stats['win_rate']*100:.1f}%, Loss: {stats['loss_rate']*100:.1f}%, Draws: {stats['draw_rate']*100:.1f}%")

# === Bar Plot Summary ===
labels = list(evaluation_results.keys())
win_rates = [evaluation_results[k]['win_rate'] * 100 for k in labels]
loss_rates = [evaluation_results[k]['loss_rate'] * 100 for k in labels]
draw_rates = [evaluation_results[k]['draw_rate'] * 100 for k in labels]

x = range(len(labels))
bar_width = 0.25

plt.figure(figsize=(12, 6))
plt.bar(x, win_rates, width=bar_width, label='Win %', color='green')
plt.bar([i + bar_width for i in x], loss_rates, width=bar_width, label='Loss %', color='red')
plt.bar([i + 2 * bar_width for i in x], draw_rates, width=bar_width, label='Draw %', color='gray')

plt.xlabel('Opponent Type')
plt.ylabel('Percentage')
plt.title('DQN Agent Performance vs Various Opponents')
plt.xticks([i + bar_width for i in x], labels, rotation=15)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Convert evaluation results to DataFrame
df_eval = pd.DataFrame.from_dict(evaluation_results, orient='index')
df_eval.index.name = "Opponent"

# Save to CSV
df_eval.to_excel(f"{LOG_DIR}DQN-{TRAINING_SESSION}-evaluation_results.xlsx", index=True)


# DONE

In [ ]:
total_end_time = time.time()
total_elapsed = (total_end_time - begin_start_time) / 3600
print(f"Evaluation completed in {total_elapsed:.1f} hours")

## Training log

In [ ]:
# TRAINING_SESSION

training_log_file = "DQN training_sessions.xlsx"
log_row = {"TRAINING_SESSION": TRAINING_SESSION, "TIME [h]": total_elapsed, "EPISODES": num_episodes}

for label, stats in evaluation_results.items():
    log_row[label] = stats["win_rate"]

# === Load or Create Excel File ===
if os.path.exists(training_log_file):
    df_log = pd.read_excel(training_log_file)
else:
    df_log = pd.DataFrame()

# === Append and Save ===
df_log = pd.concat([df_log, pd.DataFrame([log_row])], ignore_index=True)
df_log.to_excel(training_log_file, index=False)

print(f"\n📁 Training session logged to: {training_log_file}")